In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('drive/MyDrive/Blindness Detection/train.csv')

In [4]:
# Split the data into training (70%), validation (15%), and test (15%)
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


In [5]:
# Define the root directory where your data is stored
root_data_dir = 'drive/MyDrive/Blindness Detection/gaussian_filtered_images'

In [6]:
# Create directories for each split and class
splits = ['train', 'valid', 'test']
classes = ["Mild", "Moderate", "No_DR", "Proliferate_DR", "Severe"]

In [ ]:
for class_name in classes:
    # Create directories for each split (train, valid, test) for the current class
    for split in splits:
        os.makedirs(os.path.join(root_data_dir, split, class_name), exist_ok=True)

    # Get the list of image files for the current class
    image_files = os.listdir(os.path.join(root_data_dir, class_name))

    # Split the image files into train, valid, and test sets
    train_files, valid_test_files = train_test_split(image_files, test_size=0.3, random_state=42)
    valid_files, test_files = train_test_split(valid_test_files, test_size=0.5, random_state=42)

    # Move files to their respective directories
    for file in train_files:
        src_path = os.path.join(root_data_dir, class_name, file)
        dest_path = os.path.join(root_data_dir, 'train', class_name, file)
        shutil.move(src_path, dest_path)

    for file in valid_files:
        src_path = os.path.join(root_data_dir, class_name, file)
        dest_path = os.path.join(root_data_dir, 'valid', class_name, file)
        shutil.move(src_path, dest_path)

    for file in test_files:
        src_path = os.path.join(root_data_dir, class_name, file)
        dest_path = os.path.join(root_data_dir, 'test', class_name, file)
        shutil.move(src_path, dest_path)

print("Dataset splitting completed.")

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [8]:
# Define the paths to your train, validation, and test directories
train_dir = 'drive/MyDrive/Blindness Detection/gaussian_filtered_images/train'
valid_dir = 'drive/MyDrive/Blindness Detection/gaussian_filtered_images/valid'
test_dir = 'drive/MyDrive/Blindness Detection/gaussian_filtered_images/test'

In [9]:
# Define image dimensions and batch size
image_size = (224, 224)
batch_size = 32

In [10]:
# Create data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,      # Rescale pixel values to [0, 1]
    rotation_range=20,        # Randomly rotate images
    width_shift_range=0.2,    # Randomly shift images horizontally
    height_shift_range=0.2,   # Randomly shift images vertically
    horizontal_flip=True,     # Randomly flip images horizontally
    shear_range=0.2,          # Shear transformations
    zoom_range=0.2            # Randomly zoom in on images
)

# Create data generators without data augmentation for validation and test sets
valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)


In [11]:
# Load and augment the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # for multi-class classification
)

# Load the validation data (no augmentation)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the test data (no augmentation)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2562 images belonging to 5 classes.
Found 549 images belonging to 5 classes.
Found 551 images belonging to 5 classes.


In [12]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [13]:
# Add custom layers for your specific classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # 5 classes for Diabetic Retinopathy


In [14]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained VGG16 model (optional)
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=valid_generator
)

Epoch 1/10
81/81 [==============================] - 901s 11s/step - loss: 1.0184 - accuracy: 0.6358 - val_loss: 0.9114 - val_accuracy: 0.6721
Epoch 2/10
81/81 [==============================] - 46s 567ms/step - loss: 0.8851 - accuracy: 0.6815 - val_loss: 0.8683 - val_accuracy: 0.6812
Epoch 3/10
81/81 [==============================] - 48s 597ms/step - loss: 0.8705 - accuracy: 0.6889 - val_loss: 0.8048 - val_accuracy: 0.7067
Epoch 4/10
81/81 [==============================] - 46s 568ms/step - loss: 0.8667 - accuracy: 0.6870 - val_loss: 0.8108 - val_accuracy: 0.7122
Epoch 5/10
81/81 [==============================] - 48s 594ms/step - loss: 0.8426 - accuracy: 0.7018 - val_loss: 0.8206 - val_accuracy: 0.7122
Epoch 6/10
81/81 [==============================] - 46s 568ms/step - loss: 0.8051 - accuracy: 0.7108 - val_loss: 0.7770 - val_accuracy: 0.7177
Epoch 7/10
81/81 [==============================] - 47s 585ms/step - loss: 0.8065 - accuracy: 0.7131 - val_loss: 0.7829 - val_accuracy: 0.7213


In [17]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

18/18 [==============================] - 189s 11s/step - loss: 0.8704 - accuracy: 0.6770
Test Accuracy: 67.70%
